# Inspect Training Data Generator Logic

### Import

In [1]:
import sys
sys.path.append('source')
from training_data_generator import TrainingDataGenerator

### Initialise

In [2]:
generator = TrainingDataGenerator(
    'question_bank.md',
    'data/trees_df.csv',
    'data/patches_df.csv'
)

### Load Data

In [3]:
generator.load_data()

### Check Templates

In [4]:
generator.parse_question_bank()

{'tree_recommendations_(generic)': {'fill_with': 'level numbers from trees_df.level_requirement',
  'templates': ['What should I grow at level ___?',
   'What should I plant at level ___?',
   'Which are the best trees at level ___?']},
 'tree_recommendations_(regular)': {'fill_with': 'level numbers from trees_df.level_requirement',
  'templates': ['What is the best tree at level ___?',
   'Which tree should I grow at level ___?',
   'What tree should I plant at level ___?',
   'What tree is best at level ___?']},
 'tree_recommendations_(fruit)': {'fill_with': 'level numbers from trees_df.level_requirement',
  'templates': ['What is the best fruit tree at level ___?',
   'Which fruit tree should I grow at level ___?',
   'What fruit tree should I plant at level ___?',
   'What fruit tree is best at level ___?']},
 'level_requirements': {'fill_with': 'tree names from trees_df.tree_name',
  'templates': ['What level do I need for ___ trees?',
   'What level do I need to plant ___?',
   '

### Check Fill Values

In [5]:
print(generator.get_fill_values('level numbers'))
print(generator.get_fill_values('tree names'))
print(generator.get_fill_values('patch names'))

['15', '27', '30', '33', '39', '42', '45', '51', '57', '60', '68', '75', '81']
['oak', 'willow', 'maple', 'yew', 'magic', 'apple', 'banana', 'orange', 'curry', 'pineapple', 'papaya', 'palm', 'dragonfruit']
['lumbridge', 'varrock', 'falador', 'taverley', 'gnome stronghold', 'farming guild', 'nemus retreat', 'gnome stronghold', 'catherby', 'tree gnome village', 'brimhaven', 'lletya', 'farming guild', 'kastori']


# Generate Training Data

In [7]:
import pandas as pd
df = generator.generate_dataset()